# transformation of the data available in the Google sheet "ODP OPENNESS INDICATOR_local"

## connect to google drive API and read the data in the file

 gspread [makes possible to access google sheets](https://gspread.readthedocs.io/en/latest/)

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from tqdm.notebook import tnrange, tqdm

In [2]:
def set_google_drive_access_scope():
    """
    define the scope of the access to google drive and create credentials using the privatepprint.json file)
    """
    
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

    creds = ServiceAccountCredentials.from_json_keyfile_name('dataset_types_ODP.json', scope)

    client = gspread.authorize(creds)
    
    return client

## get the data from the google sheet

In [3]:
def access_google_sheet(sheet_name, worksheet_name, client):
    """
    getting access to the google sheet spreadsheet and the name of the "History" sheet we want to access
    """
    
    sheet = client.open(sheet_name) # access the google sheet
    history = sheet.worksheet(worksheet_name) # access the worksheet "History"
    
    # extract all the raw data available
    data_types = history.get_all_records()
    
    return data_types

## tranforming the data into tabular data and making some cleaning

In [4]:
def transform_into_dataframe(data):
    """
    transform the dict into a padas
    """
    
    raw_data = pd.DataFrame(data)    
    
    return raw_data

## making some data cleaning

In [5]:
def drop_not_used_columns(data):
    """
    drop not required columns: openess indicator label, openess indicator value, total datasets label, total datasets
    """
    
    return data.drop(["2", "3", "4", "5"], axis=1)

# processing each row of the dataset

In [6]:
def extract_row(row_to_process, data):
    """
    extract the information correspondig to a specific row for reshape it properly: <date>, <data type>, <number of datasets>
    """   
    data_row = data.loc[row_to_process]
    
    #create a dataframe with the information extacter for each row. It will make further reshaping easier
    return pd.DataFrame(data_row).T.reset_index().drop(["index"], axis=1)

In [7]:
def row_removing_empty_columns(data_row):
    """
    remove the columns having no values. Here no value is represented by ''
    """

    return data_row.drop([col for col in data_row if (data_row[col] == '').any()], axis=1)

In [8]:
def columns_identification(data_row):
    """
    extract the columns corresponding to the data types and the number of datasets per data type
    """

    data_row_type_labels = data_row.columns[1:data_row.shape[1]:2]
    data_row_value_labels = data_row.columns[2:data_row.shape[1]:2]
    
    return data_row_type_labels, data_row_value_labels

In [9]:
def columns_separation(data_row, data_row_type_labels, data_row_value_labels):
    """
    create 2 dataframes: 1 containing the data set type and 1 containing the value of the corresponding dataset type and reshape as row-oriented by transposing it
    """
    # dataset type columns extraction
    data_row_type = data_row[data_row_type_labels].T.reset_index()
    data_row_type = data_row_type.drop(["index"], axis=1).rename(columns={0:"dataset type"}) # rename the column name
    
    # number of dataset types columns extraction
    data_row_value = data_row[data_row_value_labels].T.reset_index()
    data_row_value = data_row_value.drop(["index"], axis=1).rename(columns={0:"number of datasets"}) # rename the column name
    
    return data_row_type, data_row_value

In [10]:
def clean_dataset_types_description(data_row_type):
    """
    clean the row_data_type by removing unwanted text: text before the "/" character. It also removes any rows with 'None' value
    """

    data_row_type = data_row_type.dropna() #remove rows having 'None' values
    data_row_type_cleaned = data_row_type["dataset type"].apply(lambda x: x.lower()).apply(lambda x: x.split("/"))
    data_row_type_cleaned = pd.DataFrame(data_row_type_cleaned)
    data_row_type_cleaned = data_row_type_cleaned["dataset type"].apply(lambda x: x[-1])
    
    return pd.DataFrame(data_row_type_cleaned)

In [11]:
def merge_columns(data_row_type, data_row_value):
    """
    concatenate the 2 created dataframes into 1 that will serve in the final datamodel for Qlik Sense
    """
    
    data_row_transformed = pd.concat([data_row_type, data_row_value], axis=1, ignore_index=True)

    # rename the columns
    data_row_transformed.rename(columns={0:"dataset type", 1:"number of datasets"}, inplace=True)
    
    return data_row_transformed

# equivalences table for removing duplicated datasets: same dataset type, different names

In [12]:
def load_equivalences_table():
    """
    table creation by manually entry the possible datasets types writting: added as founded in the dataset
    """

    data_types_equivalance = {"comma-separated-values":"csv", "sparql-query":"sparql", "tab-separated-values":"tsv", "pdf;type=pdf1x":"pdf", "rdf+xml":"rdf_xml"}
    
    return data_types_equivalance

In [13]:
def find_and_replace_equivalent_dataset_name(data_row_transformed, data_types_equivalance):
    """
    find and replace dataset types names to remove equivalent names for the same types of datasets
    """
    
    data_row_transformed.replace(data_types_equivalance, inplace=True)
    
    return data_row_transformed
    

In [14]:
def data_grouping(data_row_transformed):
    """
    removing duplicated data: group by dataset type and summing up the number of datasets
    """
    
    return data_row_transformed.groupby(["dataset type"]).sum().reset_index().rename(columns={1:"number of datasets"})

## add the date column to finalise the datamodel

In [15]:
def remove_hours(data_row):
    """
    remove the hours minutes and seconds in the date dimention
    """

    date_object = datetime.datetime.strptime(data_row["1"][0], '%m/%d/%Y %H:%M:%S').date()

    # transform the datetime object into a string with format dd/mm/yyy
    date = date_object.strftime('%d/%m/%Y')

    return date

In [16]:
def track_proccesed_dates(dates_proccesed, current_date):
        """
        create a list of already processed dates from the google sheet file containing the data source. This list will be saved on a file for reuse when updating the processed data
        """
        
        return dates_proccesed.append({"processed dates": current_date}, ignore_index=True)

In [17]:
def month_change_detector(data_cleaned, current_date): 
    """
    detect a month change in the dataset and add a flag to the rows of the last day of each month (flag = 1) otherwise, there is not flag (flag = 0).
    
    The current dataset exhibits a non-continuity of the dates, there are many days missing.
    """
      
    # extract the previous date to check if the month has changed
    past_date = date_tracking.loc[current_row - 1][0]    

    # transform the dates into a datetime object
    past_date_datetime = datetime.datetime.strptime(past_date, "%d/%m/%Y")
    current_date_datetime = datetime.datetime.strptime(current_date, "%d/%m/%Y")

    # extract the month of the date
    past_month = past_date_datetime.month
    current_month = current_date_datetime.month

    # detect month change
    if current_month != past_month:
        flag_month_change = 1
    else:
        flag_month_change = 0
        
    return flag_month_change, past_date, current_date

In [18]:
def date_formatting(length_data_row, date):
    """
    create a dataframe with the date having equal length that row_data_type_labels and  row_data_value_labels
    """
    
    date_list = [[date, 0] for i in range(length_data_row)]
           
    return pd.DataFrame(date_list)

In [19]:
def row_data_merge(df_date, data_row_transformed):
    """
    for the current processing row: add the date colum to the dataframe containing the cleaned version of the data types and the dataset number per type
    """

    data_row_transformed = pd.concat([df_date, data_row_transformed], axis=1, ignore_index=True)

    # rename the columns, sort values by number of datasets type and remove a self-created "index" column
    data_row_transformed.rename(columns={0:"date", 1:"last day of the month", 2:"dataset type", 3:"number of datasets"}, inplace=True)
    data_row_transformed = data_row_transformed.sort_values(by=["number of datasets"], ascending=False).reset_index().drop(["index"], axis=1)
    
    return data_row_transformed

In [20]:
def add_flag_for_month_change(data_processed, past_date):
    """
    add a flag in case of month change detected - month_change_flag = 1    
    """
    
    # add a flag=1 to a previous date if there is month change
    data_processed.loc[data_processed["date"] == past_date, "last day of the month"] = 1
    
    return data_processed

## save the data and the last update info

In [25]:
def save_data_and_info_update(data_processed, date_processed): 
    """
    save the processed data into a CSV file: datasets_formats_processed.csv - headers included
    save the processed dates for start next updates from this position and not the entire data file. It stores the information into a CSV file: processed_dates_update.csv - headers included
    
    IMPORTANT: put attention to the date format in the "processed_dates_update.csv" file. The format doesn't match the format used in google sheets. You will need to change the date format in this file before
                going for updates
    """

    # save the processed data into a CSV file with headers
#     file_path_data = "C:\\Users\\carlo\\Dropbox\\Programming\\Python\\dataset types ODP\\datasets_formats_processed.csv"  # my office laptop
    file_path_data = "D:\\Dropbox\\Programming\\Python\\datasets files formats ODP\\datasets_formats_processed.csv"  # my home laptop
    data_processed.to_csv(file_path_data, index=False, header=True)
    
    # save the last row processed info (google sheet row number and last date present in this row) into a CSV file with headers
#     file_path_update = "C:\\Users\\carlo\\Dropbox\\Programming\\Python\\dataset types ODP\\processed_dates_update.csv"  # my office laptop
    file_path_update =  "D:\\Dropbox\\Programming\\Python\\datasets files formats ODP\\processed_dates_update.csv"  # my home laptop
    date_processed.to_csv(file_path_update, index=False, header=True)
    
    return

## --> main function <--

In [26]:
# global variables
sheet = "ODP OPENNESS INDICATOR_local_4"
worksheet = "History"

# set connection to google drive
google_client = set_google_drive_access_scope()

# acquire the data
data_types = access_google_sheet(sheet, worksheet, google_client)
raw_data = transform_into_dataframe(data_types)

total_rows = raw_data.shape[0]
print(f'the size of the imported data is: {raw_data.shape}\n')

# clean the data
data_cleaned = drop_not_used_columns(raw_data)

# load the table containing equivalent names for the same dataset. It'll be used to have the same name for the same dataset type
datasets_type_equivalences = load_equivalences_table()

# process all the rows of the dataset
# total_rows = 10
for current_row in tqdm(range(total_rows), desc="data rows processing"): #tqdm_notebook
    # process each row
    row_data = extract_row(current_row, data_cleaned)

    # remove empty columns in the extracted row
    row_data_clean = row_removing_empty_columns(row_data)

    # identification of the columns related to the dataset formats and the columns related to the number of dataset formats
    row_data_type_labels, row_data_value_labels = columns_identification(row_data_clean)

    # separation of the columns related to the dataset formats and the columns related to the number of dataset formats
    row_data_type, row_data_value = columns_separation(row_data_clean, row_data_type_labels, row_data_value_labels)

    # clean the dataset type desciption column by filtering out the dataset type
    row_data_type_clean = clean_dataset_types_description (row_data_type)

    # merge dataset types and dataset values into a single dataframe
    row_data_transformed = merge_columns(row_data_type_clean, row_data_value)    

    # find and replace equivalent names for the same dataset type
    row_data_transformed_cleaned = find_and_replace_equivalent_dataset_name(row_data_transformed, datasets_type_equivalences)

    # combine together same dataset types and sum up the number of datasets per data type
    row_data_transformed_cleaned = data_grouping(row_data_transformed_cleaned)

    # remove hour info from the date
    date_clean = remove_hours(row_data)
    
    # add the current date to the list of already processed dates
    if current_row == 0:
        date_tracking = pd.DataFrame({"processed dates": [date_clean]})        
    else:
        date_tracking = track_proccesed_dates(date_tracking, date_clean)
        # detect a change of month 
        month_change_flag, past_date, current_date = month_change_detector(date_tracking, date_clean)      
                   
    # generate a new date column with a month change flag column ready to add to the cleaned row dataset
    df_date = date_formatting(row_data_transformed_cleaned.shape[0], date_clean)
    
    # add the date info to the dataframe containing he dataset types and the number of dataset types and make some formatting
    row_data_final = row_data_merge(df_date, row_data_transformed_cleaned)

    # append the processed data
    if current_row == 0:
        data_processed = row_data_final
    else:
        data_processed = data_processed.append(row_data_final, ignore_index=True)
        # change the "last day of the month" column vaue to 0 --> 1
        if month_change_flag:
            data_processed = add_flag_for_month_change(data_processed, past_date)
        
# save the processed data into a CSV file
save_data_and_info_update(data_processed, date_tracking)


the size of the imported data is: (1000, 104)



# --> dev zone

In [242]:
# process all the rows of the dataset
total_rows = 1038 #1038
for current_row in tqdm(range(total_rows), desc="data rows processing"): #tqdm_notebook
    # process each row
    row_data = extract_row(current_row, data_cleaned)

    # remove empty columns in the extracted row
    row_data_clean = row_removing_empty_columns(row_data)

    # identification of the columns related to the dataset formats and the columns related to the number of dataset formats
    row_data_type_labels, row_data_value_labels = columns_identification(row_data_clean)

    # separation of the columns related to the dataset formats and the columns related to the number of dataset formats
    row_data_type, row_data_value = columns_separation(row_data_clean, row_data_type_labels, row_data_value_labels)

    # clean the dataset type desciption column by filtering out the dataset type
    row_data_type_clean = clean_dataset_types_description (row_data_type)

    # merge dataset types and dataset values into a single dataframe
    row_data_transformed = merge_columns(row_data_type_clean, row_data_value)    

    # find and replace equivalent names for the same dataset type
    row_data_transformed_cleaned = find_and_replace_equivalent_dataset_name(row_data_transformed, datasets_type_equivalences)

    # combine together same dataset types and sum up the number of datasets per data type
    row_data_transformed_cleaned = data_grouping(row_data_transformed_cleaned)

    # remove hour info from the date
    date_clean = remove_hours(row_data)
    
    # add the current date to the list of already processed dates
    if current_row == 0:
        date_tracking = pd.DataFrame({"processed dates": [date_clean]})        
    else:
        date_tracking = track_proccesed_dates(date_tracking, date_clean)
        # detect a change of month 
        month_change_flag, past_date, current_date = month_change_detector(date_tracking, date_clean)
                        
        if month_change_flag:
            print(f'month change flag = {month_change_flag} previous date: {past_date} current date: {current_date}')    
                   
    # generate a new date column with a month change flag column ready to add to the cleaned row dataset
    df_date = date_formatting(row_data_transformed_cleaned.shape[0], date_clean)
    
    # add the date info to the dataframe containing he dataset types and the number of dataset types and make some formatting
    row_data_final = row_data_merge(df_date, row_data_transformed_cleaned)

    # append the processed data
    if current_row == 0:
        data_processed = row_data_final
    else:
        data_processed = data_processed.append(row_data_final, ignore_index=True)
        # change the "last day of the month" column vaue to 0 --> 1
        if month_change_flag:
            data_processed = add_flag_for_month_change(data_processed, past_date)
        
# save the processed data into a CSV file
save_data_and_info_update(data_processed, date_tracking)


month change flag = 1 previous date: 31/07/2016 current date: 01/08/2016
month change flag = 1 previous date: 29/08/2016 current date: 06/09/2016
month change flag = 1 previous date: 30/09/2016 current date: 01/10/2016
month change flag = 1 previous date: 31/10/2016 current date: 01/11/2016
month change flag = 1 previous date: 30/11/2016 current date: 01/12/2016
month change flag = 1 previous date: 31/12/2016 current date: 01/01/2017
month change flag = 1 previous date: 31/01/2017 current date: 01/02/2017
month change flag = 1 previous date: 28/02/2017 current date: 01/03/2017
month change flag = 1 previous date: 30/03/2017 current date: 10/04/2017
month change flag = 1 previous date: 30/04/2017 current date: 01/05/2017
month change flag = 1 previous date: 31/05/2017 current date: 01/06/2017
month change flag = 1 previous date: 30/06/2017 current date: 01/07/2017
month change flag = 1 previous date: 31/07/2017 current date: 01/08/2017
month change flag = 1 previous date: 31/08/2017 cur